# Explore source measurement on a custom coadd

**Runs at:** data-int.lsst.cloud

**Last run with:**

In [1]:
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

Weekly 2022_35
lsst_distrib          g0b29ad24fb+434521fcbd 	w_2022_35 current setup


<br>

**Goal:** For a custom i-band coadd that I have made with just a week's worth of inputs:
 * (1) Run source detection and measurement on the coadd.
 * (2) Do forced photometry at a location where there was no detection.
 
Then, all this will become part of Section 5 of "draft_create_custom_coadds.ipynb".

**Past Errors**

(1) The fact that both `mergeDetections` and `mergeMeasurements` require the `priorityList` to be set to `i` was figured out from error messages returned when spe is defined.

(2) Now fixed by Weekly 2022_35:

> lsst.pipe.base.pipeline ERROR: Configuration validation failed for task measure (lsst.pipe.tasks.multiBand.MeasureMergedCoaddSourcesTask)

> ValueError: Gen2 (cal_ref_cat) and Gen3 (ref_cat) reference catalogs are different.  These options must be kept in sync until Gen2 is retired.

 - ~ This was stemming from multiBand.py, MeasureMergedCoaddSourcesConfig.validate(self)
 - ~ https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/multiBand.py#L824
 - ~ This error went away when I used Weekly 2022_35

**New Error** 

There's a new ref_cat related error when defining spe. 

> LookupError: DatasetType 'ref_cat' referenced by MeasureMergedCoaddSourcesConnections uses 'skypix' as a dimension placeholder, but does not already exist in the registry.  Note that reference catalog names are now used as the dataset type name instead of 'ref_cat'.

This 'ref_cat' parameter doesn't seem to be something I can set; as far as I can tell it's not a configuration I can set like `priorityList`.
The error message says to use `MeasureMergedCoaddSourcesConfig.inputCatalog` instead of `MeasureMergedCoaddSourcesConnections.defaultTemplates`, but I'm not sure how the user is choosing one or the other.

The part of the error message that says "does not already exist in the registry" made me think that maybe I need a reference catalog collection to be included when I define simpleButler, but neither of the following were helpful:<br>
`collections = [collection,'2.2i/refcats/ci_imsim']`<br>
`collections = [collection,'refcats']`<br>


<br>

**Imports & Set Up**

In [2]:
import time
import numpy as np
import pandas

import lsst.geom
import lsst.afw.display as afwDisplay

from lsst.daf.butler import Butler

# lsst packages for executing pipeline tasks
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, Instrument

In [3]:
config = "dp02"
collection = "u/melissagraham/coadd_recreation_nb"
outputRun = "u/melissagraham/coadd_recreation_nb/TestWindow1"
my_dataId = {'band': 'i', 'tract': 4431, 'patch': 17}

<br>

**Optional:** Get the coadd I made in draft_Create_Custom_Coadd.ipynb

In [4]:
# butler = Butler(config, collections=collection)
# my_coadd = butler.get('deepCoadd', dataId=my_dataId)

In [5]:
# my_coadd_inputs = butler.get("deepCoadd.coaddInputs", my_dataId)
# my_coadd_inputs.visits.asAstropy()

In [6]:
# del butler, my_coadd, my_coadd_inputs

<br>

**simpleButler**

In [7]:
simpleButler = Butler(config, collections=[collection], run=outputRun, writeable=True)

In [8]:
simpleButler.registry.getCollectionChain(collection)

CollectionSearch(('u/melissagraham/coadd_recreation_nb/TestWindow1', '2.2i/runs/DP0.2'))

In [9]:
### what kind of refcats are there
# for c in sorted(simpleButler.registry.queryCollections()):
#     if c.find('ref') > -1:
#         print(c)

<br>

**Set up the pipeline.**

In the DRP.yaml file, the source detection steps that come after assembleCoadd are:
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask
 - detection: lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask
 - mergeDetections: lsst.pipe.tasks.mergeDetections.MergeDetectionsTask
 - deblend: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiTask
 - measure: lsst.pipe.tasks.multiBand.MeasureMergedCoaddSourcesTask
 - mergeMeasurements: lsst.pipe.tasks.mergeMeasurements.MergeMeasurementsTask
 - writeObjectTable: lsst.pipe.tasks.postprocess.WriteObjectTableTask
 
Not needed?
 - healSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.HealSparsePropertyMapTask
 - consolidateHealSparsePropertyMaps: lsst.pipe.tasks.healSparseMapping.ConsolidateHealSparsePropertyMapTask

Starting with `detection` and all the lists tasks after it.

**Which yaml file?**

Initially we used `${PIPE_TASKS_DIR}/pipelines/DRP.yaml`, but this points to `$DRP_PIPE_DIR/ingredients/DRP-full.yaml`.

The `$DRP_PIPE_DIR/ingredients/` directory actually contains `DRP-full.yaml`, which does not list any steps past measure, and `DRP-minimal-calibration.yaml`, which does have the steps we're interested in. So we use the latter.

In [10]:
# yaml_file = '${PIPE_TASKS_DIR}/pipelines/DRP.yaml'
# yaml_file = '$DRP_PIPE_DIR/ingredients/DRP-full.yaml'
yaml_file = '$DRP_PIPE_DIR/ingredients/DRP-minimal-calibration.yaml'

steps = 'detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable'
my_uri = yaml_file + '#' + steps

print(my_uri)

$DRP_PIPE_DIR/ingredients/DRP-minimal-calibration.yaml#detection,mergeDetections,deblend,measure,mergeMeasurements,writeObjectTable


In [11]:
measureSourcesPipeline = Pipeline.from_uri(my_uri)

<br>

**query string**

In [12]:
queryString = "tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'"
print(queryString)

tract = 4431 AND patch = 17 AND band = 'i' AND skymap = 'DC2'


<br>

**Configurations**

In [13]:
measureSourcesPipeline.addConfigOverride('mergeDetections', 'priorityList', 'i')
measureSourcesPipeline.addConfigOverride('mergeMeasurements', 'priorityList', 'i')

Examine configs.

configs for measure -- why does the measure_config produce errors?

In [14]:
# my_measure_config = simpleButler.get("measure_config", my_dataId)
# for key, value in my_measure_config.items():
#     print(key, value)

configs for mergeMeasurements

In [15]:
my_mergeMeasurements_config = simpleButler.get("mergeMeasurements_config", my_dataId)
for key, value in my_mergeMeasurements_config.items():
    print(key, value)

saveMetadata True
saveLogOutput True
pseudoFilterList ['sky']
snName base_PsfFlux
minSN 10.0
minSNDiff 3.0
flags ['base_PixelFlags_flag_interpolatedCenter', 'base_PsfFlux_flag', 'ext_photometryKron_KronFlux_flag', 'modelfit_CModel_flag']
priorityList ['i', 'r', 'z', 'y', 'g', 'u']
coaddName deep
connections {'inputSchema': '{inputCoaddName}Coadd_meas_schema', 'outputSchema': '{outputCoaddName}Coadd_ref_schema', 'catalogs': '{inputCoaddName}Coadd_meas', 'mergedCatalog': '{outputCoaddName}Coadd_ref', 'inputCoaddName': 'deep', 'outputCoaddName': 'deep'}


configs for writeObjectTable

In [16]:
# my_writeObjectTable_config = simpleButler.get("writeObjectTable_config", my_dataId)
# for key, value in my_writeObjectTable_config.items():
#     print(key, value)

Attempt to find configs for MeasureMergedCoaddSources, which seems to be the root of the errors from the next cell.

This does not work.

In [17]:
# temp_config = simpleButler.get("MeasureMergedCoaddSources_config", my_dataId)
# for key, value in temp_config.items():
#     print(key, value)

<br>

**Create SPE**

In [18]:
%%time
spe = SimplePipelineExecutor.from_pipeline(measureSourcesPipeline, where=queryString, butler=simpleButler)

/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pipe_tasks/gb1bf645dd1+9231f36432/python/lsst/pipe/tasks/multiBand.py:243: UserWarning: MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. Use MeasureMergedCoaddSourcesConfig.inputCatalog.
  warnings.warn("MeasureMergedCoaddSourcesConnections.defaultTemplates is deprecated and no longer used. "
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pex_config/g849534e15f+3b870f08dc/python/lsst/pex/config/configurableField.py:79: FutureWarning: Call to deprecated class LoadIndexedReferenceObjectsConfig. (This config is no longer used; it will be removed after v25. Please use LoadReferenceObjectsConfig instead.) -- Deprecated since version v25.0.
  value = self._ConfigClass(__name=name, __at=at, __label=label, **storage)
/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/pex_config/g849534e15f+3b870f08dc/python/lsst/pex/config/configurableField.py:359: Futur

Overriding default configuration file with /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/dustmaps_cachedata/g41a3ec361e+c9b10be330/config/.dustmapsrc


LookupError: DatasetType 'ref_cat' referenced by MeasureMergedCoaddSourcesConnections uses 'skypix' as a dimension placeholder, but does not already exist in the registry.  Note that reference catalog names are now used as the dataset type name instead of 'ref_cat'.

In [ ]:
del simpleButler

<br><br><br><br><br><br><br>


# earlier attempts. ignore below


## 2. Characterize Image Task

> **Help Question:** Is it necessary to run CharacterizeImageTask on the coadd? It seems the image already has psf, is already characterized?

In [ ]:
my_coadd_bbox = butler.get("deepCoadd.bbox", dataId=my_dataId)
x_val = my_coadd_bbox.beginX + 200
y_val = my_coadd_bbox.beginY + 200
point = lsst.geom.Point2D(x_val, y_val)

psf = my_coadd.getPsf()
psfShape = psf.computeShape(point)
sigma = psfShape.getDeterminantRadius()
pixelScale = my_coadd.getWcs().getPixelScale().asArcseconds()
print('psf fwhm = {:.2f} arcsec at x = {:.0f}, y = {:.0f}'.format(sigma*pixelScale*2.355,x_val,y_val))

If it's necessary, do like this?

In [ ]:
# from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

# ci_config = CharacterizeImageTask.ConfigClass()
# ci_config.psfIterations = 1
# charImageTask = CharacterizeImageTask(config=ci_config)

# result = charImageTask.run(my_coadd)

## 3. Configure Source Detection, Deblend, and Measurement Tasks

In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
tab = afwTable.SourceTable.make(schema)

In [ ]:
schema

In [ ]:
sd_config = SourceDetectionTask.ConfigClass()
sd_config.thresholdValue = 5
sd_config.thresholdType = "stdev"

sourceDetectionTask = SourceDetectionTask(schema=schema, config=sd_config)

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

In [ ]:
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
sm_config = SingleFrameMeasurementTask.ConfigClass()

sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=sm_config,
                                                   algMetadata=algMetadata)

## 4. Run Source Detection Task

In [ ]:
%%time
result = sourceDetectionTask.run(tab, my_coadd)

In [ ]:
sources = result.sources

In [ ]:
print(len(sources), result.numPosPeaks)

In [ ]:
sources.asAstropy()

In [ ]:
tx = np.where( np.isfinite( sources['coord_ra'] ) )[0]
print(len(tx))
del tx

## 5. Run Source Deblending

https://pipelines.lsst.io/modules/lsst.meas.deblender/index.html

https://pipelines.lsst.io/modules/lsst.meas.deblender/tasks/lsst.meas.deblender.SourceDeblendTask.html

> **Help Question:** `sourceDeblendTask.run` exists with an assertion error about the schemas even when we make sure the input schema matches before we start. Not sure how to fix.

SourceDeblendTask modifies the source catalog in-place.

In [ ]:
assert sources.getSchema() == schema

Attempt to fix that by redefining schema and sourceDeblendTask.

In [ ]:
schema = sources.getSchema()

In [ ]:
assert sources.getSchema() == schema

In [ ]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

Does not seem to help, below we still get the error:
> `--> 275 assert sources.getSchema() == self.schema`

In [ ]:
%%time
sourceDeblendTask.run(my_coadd, sources)

In [ ]:
print(len(sources))

## 6. Run Source Measurement

https://pipelines.lsst.io/modules/lsst.meas.base/tasks/lsst.meas.base.sfm.SingleFrameMeasurementTask.html

In [ ]:
%%time
sourceMeasurementTask.run(measCat=sources, exposure=my_coadd)